## 이미지 식별을 위한 라이브러리를 불러온다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms, models
from PIL import Image
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### 데이터 디렉토리, 분할 비율, 변환 방법을 설정한다.

In [ ]:
data_dir = '../data'
valid_size = 0.2

t_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.Resize(224),
    transforms.ToTensor()
])

### (확인) 변환 방법을 출력하여 확인해보기

In [ ]:
print(t_transforms)

## 데이터 로더 함수 작성

### (연습) trainloader와 testloader를 만들어본다.

### 1. 학습 데이터  세트 및 테스트 데이터 세트의 디렉토리 및 변환 방식을 지정한다.

In [ ]:
train_data = datasets.ImageFolder(data_dir, transform=t_transforms)
test_data = datasets.ImageFolder(data_dir, transform=t_transforms)

print(train_data)

print(len(train_data), len(test_data))

### 2. 데이터세트를 섞기 위해, 우선 인덱스를 만들어 랜덤하게 섞는다.

In [ ]:
num_train = len(train_data)
indicies = list(range(num_train))
print(indicies)

np.random.shuffle(indicies)
print(indicies)

### 3. 분할 비율(valid_size)에 따른 지점의 인덱스 값(split)을 계산한다.

In [ ]:
split = int(np.floor(num_train * valid_size))
print(split)

### 4. split을 기준으로 학습 데이터 인덱스 리스트와 테스트 인덱스 리스트로 나눈다.

In [ ]:
train_idx, test_idx = indicies[split:], indicies[:split]

print(train_idx)
print(test_idx)

### 5. 데이터 세트들의 샘플러 및 로더를 만들고 확인한다.

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=16)
testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=16)

print(trainloader.dataset.classes)
print(testloader.dataset.classes)

### 코드들을 묶어서 load_split_train_test() 함수를 만든다.

In [ ]:
def load_split_train_test(data_dir, valid_size):
    t_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.Resize(224),
        transforms.ToTensor()
    ])

    train_data = datasets.ImageFolder(data_dir, transform=t_transforms)
    test_data = datasets.ImageFolder(data_dir, transform=t_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))

    np.random.shuffle(indices)
    split = int(np.floor(num_train * valid_size))
    train_idx, test_idx = indices[split:], indices[:split]
    from torch.utils.data.sampler import SubsetRandomSampler

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=16)
    testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=16)

    return trainloader, testloader

### load_split_train_test() 함수를 이용하여 trainloader, testloader를 생성한다.

In [ ]:
trainloader, testloader = load_split_train_test(data_dir, 0.2)

print(trainloader.dataset.classes)
print(testloader.dataset.classes)

### 이미지 데이터 샘플들을 살펴본다.

임의의 데이터를 로딩한 후 이미지와 레이블을 반환하는 get_random_images() 함수를 만든다.

In [ ]:
def get_random_images(num):
    data = datasets.ImageFolder(data_dir, transform=t_transforms)
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]

    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()

    return images, labels

In [ ]:
images, labels = get_random_images(5)
to_pil = transforms.ToPILImage()
fig = plt.figure(figsize=(20, 20))
classes = trainloader.dataset.classes
for ii in range(len(images)):
    image = to_pil(images[ii])
    sub = fig.add_subplot(1, len(images), ii+1)
    index = labels[ii].item()
    sub.set_title(classes[index])
    plt.axis('off')
    plt.imshow(image)
plt.show()

### ResNet50 모델을 가져와 FCL(Fully Connected Layer)을 수정한다.

Compute device를 정한다(CPU or GPU)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

### 사전 학습된 ResNet50 모델을 지정한다.

In [ ]:
model = models.resnet50(pretrained=True)

### (확인) 수정 전의 ResNet50 모델을 확인해 본다.

In [ ]:
print(model)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(nn.Linear(2048, 512),
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(512, 2),
                         nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)
print('done!')

### (확인) FCL을 확인해 본다.

In [ ]:
print(model.fc)

### 모델의 FCL을 학습시키고 테스트 한다.

모델 학습/검증을 위한 변수를 설정한다.

In [ ]:
epochs = 10
print_every = 5
running_loss = 0
train_losses, test_losses = [], []
steps = 0

In [ ]:
for epoch in range(epochs) :
    epoch += 1
    for inputs, labels in trainloader:
        steps += 1
        print('Training step ', steps)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            print("Epoch {}/{}: ".format(epoch, epochs),
                "Train loss: {:.3f}.. ".format(running_loss/print_every),
                "Test loss: {:.3f}.. ".format(test_loss/len(testloader)),
                "Test accuracy: {:.3f}\n".format(accuracy/len(testloader)))
            running_loss = 0
            model.train()
            break

### (확인) 학습 손실값과 테스트 손실값을 그래프로 확인한다.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_losses, label='training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

### 학습/테스트 완료된 모델을 저장한다.

In [ ]:
torch.save(model, 'moonrockmodel.pth')

### 완성된 모델을 사용하여 예측한다.

저장한 모델을 불러온다.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load('moonrockmodel.pth')

### (확인) 불러온 모델을 확인해 본다.

In [ ]:
print(model)

### 이미지 예측을 위해 predict_image() 함수를 만든다.

In [ ]:
def predict_image(image):
    image_tensor = t_transforms(image).float()
    input = image_tensor.unsqueeze_(0)
    input = input.to(device)
    output = model(input)
    index = output.data.numpy().argmax()

    return index

### 5개의 이미지를 임의로 가져와 예측해 본다.

In [ ]:
model.eval()

to_pil = transforms.ToPILImage()
images, labels = get_random_images(5)
fig = plt.figure(figsize=(20, 20))
classes = trainloader.dataset.classes

for ii in range(len(images)):
    image = to_pil(images[ii])
    index= predict_image(image)
    sub = fig.add_subplot(1, len(images), ii+1)
    res = labels[ii].item() == index
    sub.set_title(classes[index] + ':' + str(res))
    plt.axis('off')
    plt.imshow(image)
plt.show()